In [1]:
# Importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style('darkgrid')
from sklearn.metrics import f1_score

# Load The Train Data

In [2]:
# Loading the train data
df = pd.read_csv('./train.csv')
X = df
y = df['Education']
for row in X.index:
    # Check for Dr. in the list
    if "Dr." in X.loc[row, "Candidate"]:
        X.loc[row, "isDoctor"] = 1
    else:
        X.loc[row, "isDoctor"] = 0
    if "Adv." in X.loc[row, "Candidate"]:
        X.loc[row, "isAdvocate"] = 1
    else:
        X.loc[row, "isAdvocate"] = 0
X = X.drop('Candidate',axis=1)
X = X.drop('Constituency ∇',axis=1)
X.drop('ID',axis=1,inplace=True)
for row in X.index:
    #Remove WhiteSpaces in State in the middle of the string.
    X.loc[row, "state"] = X.loc[row, "state"].replace(" ", "")
    # print(X.loc[row, "Total Assets"][-4:])
    if(X.loc[row, "Total Assets"][-4:] == "Lac+"):
        X.loc[row, "Total Assets"] = X.loc[row, "Total Assets"][:-4]
        X.loc[row, "Total Assets"] = str(int(X.loc[row, "Total Assets"]) * 100)
    elif(X.loc[row, "Total Assets"][-6:] == "Crore+"):
        X.loc[row, "Total Assets"] = X.loc[row, "Total Assets"][:-6]
        X.loc[row, "Total Assets"] = str(int(X.loc[row, "Total Assets"]) * 10000)
    elif(X.loc[row,  "Total Assets"][-5:] == "Thou+"):
        X.loc[row,  "Total Assets"] = X.loc[row,  "Total Assets"][:-5]
        X.loc[row,  "Total Assets"] = str(int(X.loc[row,  "Total Assets"]))
    elif(X.loc[row,  "Total Assets"][-5:] == "Hund+"):
        X.loc[row,  "Total Assets"] = "0"
for row in X.index:
    if(isinstance(X.loc[row, "Liabilities"], float)):
        X.loc[row, "Liabilities"] = "0"
    elif(X.loc[row, "Liabilities"][-4:] == "Lac+"):
        X.loc[row, "Liabilities"] = X.loc[row, "Liabilities"][:-4]
        X.loc[row, "Liabilities"] = str(int(X.loc[row, "Liabilities"]) * 100)
    elif(X.loc[row, "Liabilities"][-6:] == "Crore+"):
        X.loc[row, "Liabilities"] = X.loc[row, "Liabilities"][:-6]
        X.loc[row, "Liabilities"] = str(int(X.loc[row, "Liabilities"]) * 10000)
    elif(X.loc[row, "Liabilities"][-5:] == "Thou+"):
        X.loc[row, "Liabilities"] = X.loc[row, "Liabilities"][:-5]
        X.loc[row, "Liabilities"] = str(int(X.loc[row, "Liabilities"]))
    elif(X.loc[row, "Liabilities"][-5:] == "Hund+"):
        X.loc[row, "Liabilities"] = "0"
# Mapping Parties to unique values in the dataframe using map function
# Create a dictionary of the parties and their corresponding unique values
party = X['Party'].unique()
party = {value: idx for idx, value in enumerate(party)}
# Add a value for nan parties
party[""] = -1
# Mapping the values to the dataframe
X['Party'] = X['Party'].map(party)
state = X['state'].unique()
state = {value: idx for idx, value in enumerate(state)}
# Mapping the values to the dataframe
X['state'] = X['state'].map(state)
education = y.unique()
education = {value: idx for idx, value in enumerate(education)}
# Mapping the values to the dataframe
y = y.map(education)
X['Education'] = y

# Synthetic Data

In [3]:
# Synthetic Data 
df = pd.read_csv('./synthetic_data_best.csv')
# df = df[:1000]
X_Synthetic = df
for row in X_Synthetic.index:
	X_Synthetic.loc[row, "isDoctor"] = 0
	X_Synthetic.loc[row, "isAdvocate"] = 0
X_Synthetic['state'] = X_Synthetic['state'].map(state)
X_Synthetic['Party'] = X_Synthetic['Party'].map(party)
X_Synthetic['Education'] = X_Synthetic['Education'].map(education)
X_Synthetic

,Party,Criminal Case,Total Assets,Liabilities,state,Education,isDoctor,isAdvocate
0,5,4,1430000,10000,5,2,0.0,0.0
1,1,1,10000,600,22,2,0.0,0.0
2,1,0,10000,400,16,4,0.0,0.0
3,1,2,400,2100,19,4,0.0,0.0
4,6,0,10000,1600,10,2,0.0,0.0
...,...,...,...,...,...,...,...,...
7995,1,0,130000,0,18,4,0.0,0.0
7996,13,23,10000,1900,11,1,0.0,0.0
7997,1,0,50000,110000,4,3,0.0,0.0
7998,1,2,6900,1200,5,2,0.0,0.0


In [4]:
# Append the synthetic data to the original data
df_combined = pd.concat([X, X_Synthetic], ignore_index=True)
X = df_combined
X_Train_StateWise = []
X_Val_StateWise = []
Y_Train_StateWise = []
Y_Val_StateWise = []
# Test Val Split
for i in range(len(state)):
	X_Train_StateWise.append(X[X['state'] == i])
	if(X_Train_StateWise[i].shape[0] >=20):
		X_Val_StateWise.append(X_Train_StateWise[i].sample(frac=0.05))
		# X_Train_StateWise[i] = X_Train_StateWise[i].drop(X_Val_StateWise[i].index)
		Y_Val_StateWise.append(X_Val_StateWise[i]['Education'])
		Y_Train_StateWise.append(X_Train_StateWise[i]['Education'])
		X_Val_StateWise[i] = X_Val_StateWise[i].drop('Education', axis = 1)
		X_Train_StateWise[i] = X_Train_StateWise[i].drop('Education', axis = 1)
	else:
		X_Train_StateWise[i] = X_Train_StateWise[i]
		X_Val_StateWise.append(X_Train_StateWise[i])
		Y_Val_StateWise.append(X_Val_StateWise[i]['Education'])
		Y_Train_StateWise.append(X_Train_StateWise[i]['Education'])
		X_Val_StateWise[i] = X_Val_StateWise[i].drop('Education', axis = 1)
		X_Train_StateWise[i] = X_Train_StateWise[i].drop('Education', axis = 1)
Y = X['Education']
X = X.drop('Education', axis = 1)

# Pre-Processing

In [5]:
def test_preprocessing(X, state, party):
	for row in X.index:
		# Check for Dr. in the list
		if "Dr." in X.loc[row, "Candidate"]:
			X.loc[row, "isDoctor"] = 1
		else:
			X.loc[row, "isDoctor"] = 0
		if "Adv." in X.loc[row, "Candidate"]:
			X.loc[row, "isAdvocate"] = 1
		else:
			X.loc[row, "isAdvocate"] = 0
	X = X.drop('Candidate',axis=1)
	X = X.drop('Constituency ∇',axis=1)
	X.drop('ID',axis=1,inplace=True)
	for row in X.index:
		X.loc[row, "state"] = X.loc[row, "state"].replace(" ", "")
		# print(X.loc[row, "Total Assets"][-4:])
		if(X.loc[row, "Total Assets"][-4:] == "Lac+"):
			X.loc[row, "Total Assets"] = X.loc[row, "Total Assets"][:-4]
			X.loc[row, "Total Assets"] = str(int(X.loc[row, "Total Assets"]) * 100)
		elif(X.loc[row, "Total Assets"][-6:] == "Crore+"):
			X.loc[row, "Total Assets"] = X.loc[row, "Total Assets"][:-6]
			X.loc[row, "Total Assets"] = str(int(X.loc[row, "Total Assets"]) * 10000)
		elif(X.loc[row,  "Total Assets"][-5:] == "Thou+"):
			X.loc[row,  "Total Assets"] = X.loc[row,  "Total Assets"][:-5]
			X.loc[row,  "Total Assets"] = str(int(X.loc[row,  "Total Assets"]))
		elif(X.loc[row,  "Total Assets"][-5:] == "Hund+"):
			X.loc[row,  "Total Assets"] = "0"
	for row in X.index:
		if(X.loc[row, "Liabilities"][-4:] == "Lac+"):
			X.loc[row, "Liabilities"] = X.loc[row, "Liabilities"][:-4]
			X.loc[row, "Liabilities"] = str(int(X.loc[row, "Liabilities"]) * 100)
		elif(X.loc[row, "Liabilities"][-6:] == "Crore+"):
			X.loc[row, "Liabilities"] = X.loc[row, "Liabilities"][:-6]
			X.loc[row, "Liabilities"] = str(int(X.loc[row, "Liabilities"]) * 10000)
		elif(X.loc[row, "Liabilities"][-5:] == "Thou+"):
			X.loc[row, "Liabilities"] = X.loc[row, "Liabilities"][:-5]
			X.loc[row, "Liabilities"] = str(int(X.loc[row, "Liabilities"]))
		elif(X.loc[row, "Liabilities"][-5:] == "Hund+"):
			X.loc[row, "Liabilities"] = "0"
	X['Party'] = X['Party'].map(party)
	X['state'] = X['state'].map(state)
	return X

def answer_preprocessing(X, state, party, education):
	for row in X.index:
		# Check for Dr. in the list
		if "Dr." in X.loc[row, "Candidate"]:
			X.loc[row, "isDoctor"] = 1
		else:
			X.loc[row, "isDoctor"] = 0
		if "Adv." in X.loc[row, "Candidate"]:
			X.loc[row, "isAdvocate"] = 1
		else:
			X.loc[row, "isAdvocate"] = 0
	X = X.drop('Candidate',axis=1)
	X = X.drop('Constituency ∇',axis=1)
	X.drop('ID',axis=1,inplace=True)
	for row in X.index:
		X.loc[row, "state"] = X.loc[row, "state"].replace(" ", "")
		# print(X.loc[row, "Total Assets"][-4:])
		if(X.loc[row, "Total Assets"][-4:] == "Lac+"):
			X.loc[row, "Total Assets"] = X.loc[row, "Total Assets"][:-4]
			X.loc[row, "Total Assets"] = str(int(X.loc[row, "Total Assets"]) * 100)
		elif(X.loc[row, "Total Assets"][-6:] == "Crore+"):
			X.loc[row, "Total Assets"] = X.loc[row, "Total Assets"][:-6]
			X.loc[row, "Total Assets"] = str(int(X.loc[row, "Total Assets"]) * 10000)
		elif(X.loc[row,  "Total Assets"][-5:] == "Thou+"):
			X.loc[row,  "Total Assets"] = X.loc[row,  "Total Assets"][:-5]
			X.loc[row,  "Total Assets"] = str(int(X.loc[row,  "Total Assets"]))
		elif(X.loc[row,  "Total Assets"][-5:] == "Hund+"):
			X.loc[row,  "Total Assets"] = "0"
	for row in X.index:
		if(X.loc[row, "Liabilities"][-4:] == "Lac+"):
			X.loc[row, "Liabilities"] = X.loc[row, "Liabilities"][:-4]
			X.loc[row, "Liabilities"] = str(int(X.loc[row, "Liabilities"]) * 100)
		elif(X.loc[row, "Liabilities"][-6:] == "Crore+"):
			X.loc[row, "Liabilities"] = X.loc[row, "Liabilities"][:-6]
			X.loc[row, "Liabilities"] = str(int(X.loc[row, "Liabilities"]) * 10000)
		elif(X.loc[row, "Liabilities"][-5:] == "Thou+"):
			X.loc[row, "Liabilities"] = X.loc[row, "Liabilities"][:-5]
			X.loc[row, "Liabilities"] = str(int(X.loc[row, "Liabilities"]))
		elif(X.loc[row, "Liabilities"][-5:] == "Hund+"):
			X.loc[row, "Liabilities"] = "0"
	X['Party'] = X['Party'].map(party)
	X['state'] = X['state'].map(state)
	X['Education'] = X['Education'].map(education)
	return X

In [6]:
X_Test = test_preprocessing(pd.read_csv('./test.csv'), state, party)

X_Test_StateWise = []
Y_Test_StateWise = []
for i in range(len(state)):
    X_Test_StateWise.append(X_Test[X_Test['state'] == i])

In [7]:
from sklearn.metrics import f1_score
def calculate_accuracy(Y_Test_StateWise, answer_statewise):
	#convert this to an array
    temp = np.array(answer_statewise['Education'])
    correct_count = 0
    for i in range(len(Y_Test_StateWise)):
        if(temp[i] == Y_Test_StateWise[i]):
            correct_count += 1
    f1 = f1_score(Y_Test_StateWise, temp, average='weighted')
    return [correct_count/len(Y_Test_StateWise) * 100, correct_count, f1]

# KNN State Wise

In [8]:
#Get the neighbours from the CSV
def get_neighbours():
	f = open("parameters_KNN.txt", "r")
	neighbours_calculated = dict()
	for line in f:
		line = line.split()
		neighbours_calculated[state[line[0]]] = int(line[1])
	f.close()
	return neighbours_calculated

In [9]:
#Implement KNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

models_KNN = []
Y_Test_StateWise = []
neighbours_to_use = get_neighbours()
neighbours_calculated = dict()
f1_KNN = dict()

# Grid Search
for i in range(len(state)):
	model = KNeighborsClassifier(n_neighbors=1)
	if(len(X_Train_StateWise[i]) <= 20):
		model.fit(X_Train_StateWise[i], Y_Train_StateWise[i])
		models_KNN.append(model)
		Y_Test_StateWise.append(model.predict(X_Test_StateWise[i]))
		neighbours_calculated[i] = 1
	else:
		f1 = []
		f1_scores = 0
		# for j in range(1, 15):
		# 	model = KNeighborsClassifier(n_neighbors=j)
		# 	model.fit(X_Train_StateWise[i], Y_Train_StateWise[i])
		# 	f1_scores = (f1_score(Y_Val_StateWise[i], model.predict(X_Val_StateWise[i]),average='weighted')
		# 	f1.append(f1_score)
		neighbours_calculated[i] = neighbours_to_use[i]
		model = KNeighborsClassifier(n_neighbors=neighbours_to_use[i])
		model.fit(X_Train_StateWise[i], Y_Train_StateWise[i])
		models_KNN.append(model)
		Y_Test_StateWise.append(model.predict(X_Test_StateWise[i]))
	f1 = f1_score(Y_Val_StateWise[i], model.predict(X_Val_StateWise[i]),average='weighted')
	f1_KNN[i] = f1

In [10]:
# #Write the neighbours to a file
# f = open("parameters_KNN.txt", "w")
# reverse_state = {value: key for key, value in state.items()}
# for key in neighbours_calculated:
# 	f.write(reverse_state[key] + " " + str(neighbours_calculated[key]) + "\n")
# f.close()

In [11]:
# Print the f1 scores
reversed_state = {value: key for key, value in state.items()}
for i in range(len(state)):
	print(reversed_state[i], f1_KNN[i])

TAMILNADU 0.37213081036610446
MADHYAPRADESH 0.3261538461538461
KARNATAKA 0.5126197318007664
BIHAR 0.5101330042506514
WESTBENGAL 0.40534090909090903
UTTARPRADESH 0.46792328042328046
PUNJAB 0.5905072463768116
MAHARASHTRA 0.36966048593955575
ODISHA 0.5654024767801857
HIMACHALPRADESH 0.9090909090909093
MEGHALAYA 0.2469135802469136
KERALA 0.2213248935938012
HARYANA 0.6904761904761905
DELHI 0.3896103896103896
ANDHRAPRADESH 0.38995670995671
MANIPUR 0.7333333333333334
RAJASTHAN 0.6794871794871794
ASSAM 0.4142857142857143
ARUNACHALPRADESH 0.9090909090909091
JHARKHAND 0.325
GUJARAT 0.3894179894179894
TRIPURA 0.4571428571428572
CHHATTISGARH 0.09230769230769233
UTTARAKHAND 0.4606060606060606
NAGALAND 0.7142857142857143
SIKKIM 1.0
GOA 0.8173076923076923
PUDUCHERRY 0.6000000000000001


# DT State Wise

In [12]:
# Get the data from the CSV
def get_params():
	f = open("parameters_DT.txt", "r")
	params_calculated = dict()
	for line in f:
		line = line.split()
		param = line[1] + line [2] + line[3]
		param = param.replace('(','')
		param = param.replace(')','')
		param = param.split(',')
		param_l = (int(param[0]), int(param[1]), int(param[2]))
		params_calculated[state[line[0]]] = (param_l)
	f.close()
	return params_calculated

In [13]:
# DT State Wise
#Implement DT
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

models_DT = []
Y_Test_StateWise = []
DT_Params = dict()
DT_Params_Saved = get_params()
f1_DT = dict()

# Grid Search
for i in range(len(state)):
	model = DecisionTreeClassifier()
	if(len(X_Train_StateWise[i]) <= 20):
		model.fit(X_Train_StateWise[i], Y_Train_StateWise[i])
		models_DT.append(model)
		Y_Test_StateWise.append(model.predict(X_Test_StateWise[i]))
		DT_Params[i] = (0, 0, 0)
	else:
		f1 = dict()
		f1_scores = 0
		# for j in range(1, 20, 1):
		# 	for k in range(1, 15, 1):
		# 		for l in range(2,5):
		# 			model = DecisionTreeClassifier(max_depth=j, min_samples_leaf=k, min_samples_split=l)
		# 			model.fit(X_Train_StateWise[i], Y_Train_StateWise[i])
		# 			f1_scores = (f1_score(Y_Val_StateWise[i], model.predict(X_Val_StateWise[i]),average='weighted')
		# 			y = model.predict(X_Test_StateWise[i])
		# 			t = calculate_accuracy(y, answer_statewise[i])
		# 			f1[(j, k, l)] = t
		
		# max_params = max(f1, key=f1.get)
		# DT_Params[i] = max_params
		max_params = DT_Params_Saved[i]
		model = DecisionTreeClassifier(max_depth=max_params[0], min_samples_leaf=max_params[1], min_samples_split=max_params[2])
		model.fit(X_Train_StateWise[i], Y_Train_StateWise[i])
		models_DT.append(model)
		Y_Test_StateWise.append(model.predict(X_Test_StateWise[i]))
	f1 = f1_score(Y_Val_StateWise[i], model.predict(X_Val_StateWise[i]),average='weighted')
	f1_DT[i] = f1

In [14]:
# #Write the neighbours to a file
# f = open("parameters_DT.txt", "w")
# reverse_state = {value: key for key, value in state.items()}
# for key in DT_Params:
# 	f.write(reverse_state[key] + " " + str(DT_Params[key]) + "\n")
# f.close()

In [15]:
# Print the f1 scores
for i in range(len(state)):
	print(reversed_state[i], f1_DT[i])

TAMILNADU 0.5996983408748114
MADHYAPRADESH 0.2285714285714286
KARNATAKA 0.3445121951219512
BIHAR 0.2840909090909091
WESTBENGAL 0.724770523594053
UTTARPRADESH 0.48495842781557064
PUNJAB 0.4830434782608696
MAHARASHTRA 0.3288985647476213
ODISHA 0.5181333901555508
HIMACHALPRADESH 0.20979020979020976
MEGHALAYA 0.0808080808080808
KERALA 0.4906411049268192
HARYANA 0.7142857142857143
DELHI 0.41958041958041953
ANDHRAPRADESH 0.33752380952380956
MANIPUR 0.8174603174603173
RAJASTHAN 0.32623626373626374
ASSAM 0.06666666666666668
ARUNACHALPRADESH 0.9090909090909091
JHARKHAND 0.2654761904761905
GUJARAT 0.34920634920634924
TRIPURA 0.5142857142857143
CHHATTISGARH 0.4747252747252747
UTTARAKHAND 0.45252525252525255
NAGALAND 0.7912087912087912
SIKKIM 1.0
GOA 0.5769230769230769
PUDUCHERRY 1.0


# Final Model

In [16]:
def get_final_model():
	f = open("final_model.txt", "r")
	final_model_index = []
	for line in f:
		final_model_index.append(int(line))
	f.close()
	return final_model_index

In [17]:
final_model_index = get_final_model()
final_model = []
for i in range(len(state)):
	if(int(final_model_index[i]) == 0):
		final_model.append(models_KNN[i])
	else:
		final_model.append(models_DT[i])

In [18]:
for i in range(len(state)):
	Y_Test_StateWise[i] = final_model[i].predict(X_Test_StateWise[i])

In [19]:
Y_Final = []
for i in range(len(state)):
	Y_Final.append(X_Test[X_Test['state'] == i])


for i in range(len(state)):
	Y_Final[i]['Education'] = Y_Test_StateWise[i]

reverse_education = {value: key for key, value in education.items()}

answer_csv_to_write = []
for i in range(len(state)):
    for index, value in Y_Final[i]['Education'].items():
        answer_csv_to_write.append([index,reverse_education[value]])
answer_csv_to_write.sort()
yrt = []
for i in range(len(answer_csv_to_write)):
    yrt.append(education[answer_csv_to_write[i][1]])


/var/folders/dq/bh64zxfx1nn4xlf98w20zj0h0000gn/T/ipykernel_1267/421836279.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y_Final[i]['Education'] = Y_Test_StateWise[i]
/var/folders/dq/bh64zxfx1nn4xlf98w20zj0h0000gn/T/ipykernel_1267/421836279.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y_Final[i]['Education'] = Y_Test_StateWise[i]
/var/folders/dq/bh64zxfx1nn4xlf98w20zj0h0000gn/T/ipykernel_1267/421836279.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [20]:
with open('answer_knn_statewise_DT_KNN.csv', 'w') as f:
    f.write('ID,Education\n')
    for item in answer_csv_to_write:
        f.write(f"{item[0]},{item[1]}\n")